In [4]:
import pandas as pd
import qtl.io
from pathlib import Path

def prepare_bed(df, bed_template_df, chr_subset=None):
    bed_df = pd.merge(bed_template_df, df, left_index=True, right_index=True)
    bed_df = bed_df.groupby('#chr', sort=False, group_keys=False).apply(lambda x: x.sort_values('start'))
    if chr_subset is not None:
        bed_df = bed_df[bed_df.chr.isin(chr_subset)]
    return bed_df

def load_and_preprocess_data(input_path, drop_columns):
    df = pd.read_csv(input_path, sep="	", skiprows=0)
    dc = [col for col in df.columns if col in drop_columns] # Take interscet between df.columns and drop_columns
    df = df.drop(dc,axis = 1) # drop the intersect
    if len(df.columns) < 2:
        raise ValueError("There are too few columns in the loaded dataframe, please check the delimiter of the input file. The default delimiter is tab")
    return df

def rename_samples_using_lookup(df, lookup_path):
    sample_participant_lookup = Path(lookup_path)
    if sample_participant_lookup.is_file():
        sample_participant_lookup_s = pd.read_csv(sample_participant_lookup, sep="\t", index_col=0, dtype={0:str,1:str})
        df.rename(columns=sample_participant_lookup_s.to_dict(), inplace=True)
    return df

def load_bed_template(input_path, phenotype_id_type):
    if sum(qtl.io.gtf_to_tss_bed(input_path, feature='gene', phenotype_id = "gene_id").index.duplicated()) > 0:
        raise valueerror(f"gtf file {input_path} needs to be collapsed into gene model by reference data processing module")

    bed_template_df_id = qtl.io.gtf_to_tss_bed(input_path, feature='transcript', phenotype_id="gene_id")
    bed_template_df_name = qtl.io.gtf_to_tss_bed(input_path, feature='transcript', phenotype_id="gene_name")
    bed_template_df = bed_template_df_id.merge(bed_template_df_name, on=["chr", "start", "end"])
    bed_template_df.columns = ["#chr", "start", "end", "gene_id", "gene_name"]
    bed_template_df = bed_template_df.set_index(phenotype_id_type, drop=False)

    return bed_template_df


In [20]:
sample_participant_lookup_s = pd.read_csv("/sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/input/snmulti_QTL_sample_participant_lookup", sep="\t", index_col=0, dtype={0:str,1:str})

In [5]:

df = load_and_preprocess_data('/hpc/users/sunh14/snmulti_QTL/output/molecular_phenotype/gene_exp_dreamlet/pb4dreamlet_subclass.Astro.EC.low_expression_filtered.normalized.log2cpm.gct.gz', ["#chr","chr", "start", "end","stop","annot.seqnames","annot.start","annot.end"]) # This function will drop any columns from the DF based on the drop_columns list, so it is good to make the list comprehensive to accomodate different source of inputs.    df.set_index(df.columns[0], inplace=True)


In [27]:
sample_participant_lookup_s.to_dict()["genotype_id"]

KeyError: 'genotype_id'

In [28]:
bed_df.rename(columns=sample_participant_lookup_s.to_dict()["participant_id"], inplace=True)

In [29]:
bed_df

,#chr,start,end,ID,G-MSBB-MB000312-BR-MSBB-71973,144547,G-MSBB-MB000254-BR-MSBB-71909,G-MSBB-MB000266-BR-MSBB-71923,G-MSBB-MB000009-BR-MSBB-68427,G-MSBB-MB000016-BR-MSBB-68454,...,G-MSBB-MB000162-BR-MSBB-71803,G-MSBB-MB000036-BR-MSBB-70609,G-MSBB-MB000048-BR-MSBB-71677,G-MSBB-MB000163-BR-MSBB-71804,X579,G-MSBB-MB000278-BR-MSBB-71936,G-MSBB-MB000294-BR-MSBB-71952,G-MSBB-MB000227-BR-MSBB-71879,G-MSBB-MB000234-BR-MSBB-71886,0_MSSM_167
SLC6A13,chr12,262872,262873,ENSG00000010379,2.172658,1.709359,3.045770,-2.344371,2.504080,0.010192,...,2.256165,1.027481,0.447590,1.743385,2.046195,1.239864,2.102267,2.393921,-0.603357,-2.344370
CCDC77,chr12,389272,389273,ENSG00000120647,3.226371,3.093995,2.331558,3.317291,3.795644,3.423472,...,3.444169,3.233375,2.445654,3.093991,2.408746,2.341367,2.531974,2.759905,3.087538,2.996036
KDM5A,chr12,389319,389320,ENSG00000073614,7.058730,7.293998,6.800099,6.851682,7.050204,6.937034,...,6.898261,6.310767,6.740973,6.978000,6.790785,7.275972,7.048931,6.692078,6.962894,6.944142
B4GALNT3,chr12,459938,459939,ENSG00000139044,0.858992,0.104313,1.774543,-2.344371,1.943941,-2.344369,...,-2.344370,2.516408,0.961389,1.821125,2.608046,1.239864,0.754206,-0.508802,1.030083,1.726555
NINJ2,chr12,663778,663779,ENSG00000171840,4.546701,0.598382,0.855276,3.002470,-2.344369,1.393660,...,-2.344370,3.491366,2.445654,6.369578,5.480347,0.864375,-0.086495,-0.508802,1.958795,-2.344370
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MT-ND4,chrMT,10759,10760,ENSG00000198886,6.603008,5.736608,5.281309,5.365801,3.973807,6.126645,...,4.529717,7.275072,5.812641,7.532908,5.231229,5.512820,5.971959,5.776981,5.460123,5.852431
MT-ND5,chrMT,12336,12337,ENSG00000198786,5.983093,3.692530,4.699995,4.439164,3.072252,4.751289,...,3.444169,6.170272,5.088967,6.007528,4.395274,4.710124,3.985474,4.316632,3.305148,5.345496
MT-ND6,chrMT,14672,14673,ENSG00000198695,2.952752,1.258270,1.774543,1.105082,1.015950,2.739487,...,0.785487,4.067884,2.592432,3.062325,2.408746,1.239864,0.394246,1.902329,1.958795,2.282616
MT-CYB,chrMT,14746,14747,ENSG00000198727,5.381846,4.440262,3.045770,4.975634,3.355605,4.915208,...,2.256165,6.611748,3.767194,6.602071,4.892207,4.217086,5.603332,5.607703,4.256294,4.352466


In [7]:
df.set_index(df.columns[0], inplace=True)
df = rename_samples_using_lookup(df, "/sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/input/snmulti_QTL_sample_participant_lookup")
bed_template_df = load_bed_template('/sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/input/reference_data/Homo_sapiens.GRCh38.103.chr.reformatted.collapse_only.gene.gtf', "gene_name")
bed_df = prepare_bed(df, bed_template_df).drop("gene_name", axis=1)
bed_df = bed_df.drop_duplicates("gene_id", keep=False)
bed_df = bed_df.rename(columns={'gene_id': 'ID'})
qtl.io.write_bed(bed_df, '/sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/working/recipe_testing/data_preprocessing/Astro.EC/phenotype_data/pb4dreamlet_subclass.Astro.EC.low_expression_filtered.normalized.log2cpm.gct.bed.gz')
bed_df[["#chr","start","end","ID"]].assign(path = '/sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/working/recipe_testing/data_preprocessing/Astro.EC/phenotype_data/pb4dreamlet_subclass.Astro.EC.low_expression_filtered.normalized.log2cpm.gct.bed.gz').to_csv('/sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/working/recipe_testing/data_preprocessing/Astro.EC/phenotype_data/pb4dreamlet_subclass.Astro.EC.low_expression_filtered.normalized.log2cpm.gct.region_list.txt',"\t",index = False) 


/bin/sh: 1: bgzip: not found


BrokenPipeError: [Errno 32] Broken pipe

In [8]:
bed_df

,#chr,start,end,ID,X197,X199,X200,X202,X204,X206,...,X555,X556,X562,X577,X579,X582,X583,X586,X588,X589
SLC6A13,chr12,262872,262873,ENSG00000010379,2.172658,1.709359,3.045770,-2.344371,2.504080,0.010192,...,2.256165,1.027481,0.447590,1.743385,2.046195,1.239864,2.102267,2.393921,-0.603357,-2.344370
CCDC77,chr12,389272,389273,ENSG00000120647,3.226371,3.093995,2.331558,3.317291,3.795644,3.423472,...,3.444169,3.233375,2.445654,3.093991,2.408746,2.341367,2.531974,2.759905,3.087538,2.996036
KDM5A,chr12,389319,389320,ENSG00000073614,7.058730,7.293998,6.800099,6.851682,7.050204,6.937034,...,6.898261,6.310767,6.740973,6.978000,6.790785,7.275972,7.048931,6.692078,6.962894,6.944142
B4GALNT3,chr12,459938,459939,ENSG00000139044,0.858992,0.104313,1.774543,-2.344371,1.943941,-2.344369,...,-2.344370,2.516408,0.961389,1.821125,2.608046,1.239864,0.754206,-0.508802,1.030083,1.726555
NINJ2,chr12,663778,663779,ENSG00000171840,4.546701,0.598382,0.855276,3.002470,-2.344369,1.393660,...,-2.344370,3.491366,2.445654,6.369578,5.480347,0.864375,-0.086495,-0.508802,1.958795,-2.344370
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MT-ND4,chrMT,10759,10760,ENSG00000198886,6.603008,5.736608,5.281309,5.365801,3.973807,6.126645,...,4.529717,7.275072,5.812641,7.532908,5.231229,5.512820,5.971959,5.776981,5.460123,5.852431
MT-ND5,chrMT,12336,12337,ENSG00000198786,5.983093,3.692530,4.699995,4.439164,3.072252,4.751289,...,3.444169,6.170272,5.088967,6.007528,4.395274,4.710124,3.985474,4.316632,3.305148,5.345496
MT-ND6,chrMT,14672,14673,ENSG00000198695,2.952752,1.258270,1.774543,1.105082,1.015950,2.739487,...,0.785487,4.067884,2.592432,3.062325,2.408746,1.239864,0.394246,1.902329,1.958795,2.282616
MT-CYB,chrMT,14746,14747,ENSG00000198727,5.381846,4.440262,3.045770,4.975634,3.355605,4.915208,...,2.256165,6.611748,3.767194,6.602071,4.892207,4.217086,5.603332,5.607703,4.256294,4.352466


In [56]:
a = pd.read_csv("/sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/working/recipe_testing/data_preprocessing/Astro.EC/phenotype_data/pb4dreamlet_subclass.Astro.EC.low_expression_filtered.normalized.log2cpm.gct.bed.gz",sep = "\t",usecols=[0, 1, 2, 3])

In [44]:
b = pd.read_csv("/sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/working/recipe_testing/data_preprocessing/Astro.EC/phenotype_data/pb4dreamlet_subclass.Astro.EC.low_expression_filtered.normalized.log2cpm.gct.region_list.txt","\t")

/opt/conda/envs/TensorQTL/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3508: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


In [45]:
b

,#chr,start,end,ID,path
0,chr12,262872,262873,ENSG00000010379,/sc/arion/projects/CommonMind/roussp01a/snmult...
1,chr12,389272,389273,ENSG00000120647,/sc/arion/projects/CommonMind/roussp01a/snmult...
2,chr12,389319,389320,ENSG00000073614,/sc/arion/projects/CommonMind/roussp01a/snmult...
3,chr12,459938,459939,ENSG00000139044,/sc/arion/projects/CommonMind/roussp01a/snmult...
4,chr12,663778,663779,ENSG00000171840,/sc/arion/projects/CommonMind/roussp01a/snmult...
...,...,...,...,...,...
14450,chrMT,10759,10760,ENSG00000198886,/sc/arion/projects/CommonMind/roussp01a/snmult...
14451,chrMT,12336,12337,ENSG00000198786,/sc/arion/projects/CommonMind/roussp01a/snmult...
14452,chrMT,14672,14673,ENSG00000198695,/sc/arion/projects/CommonMind/roussp01a/snmult...
14453,chrMT,14746,14747,ENSG00000198727,/sc/arion/projects/CommonMind/roussp01a/snmult...


In [142]:
bed_template_df = load_bed_template('/sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/input/reference_data/Homo_sapiens.GRCh38.103.chr.reformatted.collapse_only.gene.gtf', "gene_id")


In [144]:
bed_template_df

,#chr,start,end,gene_id,gene_name
gene_id,,,,,
ENSG00000223972,chr1,11868,11869,ENSG00000223972,DDX11L1
ENSG00000278267,chr1,17435,17436,ENSG00000278267,MIR6859-1
ENSG00000243485,chr1,29553,29554,ENSG00000243485,MIR1302-2HG
ENSG00000227232,chr1,29569,29570,ENSG00000227232,WASH7P
ENSG00000284332,chr1,30365,30366,ENSG00000284332,MIR1302-2
...,...,...,...,...,...
ENSG00000198695,chrMT,14672,14673,ENSG00000198695,MT-ND6
ENSG00000210194,chrMT,14741,14742,ENSG00000210194,MT-TE
ENSG00000198727,chrMT,14746,14747,ENSG00000198727,MT-CYB


In [52]:
counts = b.groupby(['#chr',"path"]).size().reset_index()

In [60]:
a.merge(counts, left_on = "#chr",right_on = "#chr")

,#chr,start,end,ID,path,0
0,chr12,262872,262873,ENSG00000010379,/sc/arion/projects/CommonMind/roussp01a/snmult...,773
1,chr12,389272,389273,ENSG00000120647,/sc/arion/projects/CommonMind/roussp01a/snmult...,773
2,chr12,389319,389320,ENSG00000073614,/sc/arion/projects/CommonMind/roussp01a/snmult...,773
3,chr12,459938,459939,ENSG00000139044,/sc/arion/projects/CommonMind/roussp01a/snmult...,773
4,chr12,663778,663779,ENSG00000171840,/sc/arion/projects/CommonMind/roussp01a/snmult...,773
...,...,...,...,...,...,...
14450,chrMT,10759,10760,ENSG00000198886,/sc/arion/projects/CommonMind/roussp01a/snmult...,13
14451,chrMT,12336,12337,ENSG00000198786,/sc/arion/projects/CommonMind/roussp01a/snmult...,13
14452,chrMT,14672,14673,ENSG00000198695,/sc/arion/projects/CommonMind/roussp01a/snmult...,13
14453,chrMT,14746,14747,ENSG00000198727,/sc/arion/projects/CommonMind/roussp01a/snmult...,13


In [126]:
len(set(bed_template_df["gene_id"]))

60576

In [146]:
bed_template_df = bed_template_df[~bed_template_df["gene_id"].duplicated(keep = "last")]

In [183]:
bed_template_df[["#chr", "start", "end", "gene_id"]].to_csv("/sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/input/reference_data/Homo_sapiens.GRCh38.103.chr.reformatted.collapse_only.gene_id.unique.gtf","\t",index = False)

In [147]:
bed_template_df

,#chr,start,end,gene_id,gene_name
gene_id,,,,,
ENSG00000223972,chr1,11868,11869,ENSG00000223972,DDX11L1
ENSG00000278267,chr1,17435,17436,ENSG00000278267,MIR6859-1
ENSG00000243485,chr1,29553,29554,ENSG00000243485,MIR1302-2HG
ENSG00000227232,chr1,29569,29570,ENSG00000227232,WASH7P
ENSG00000284332,chr1,30365,30366,ENSG00000284332,MIR1302-2
...,...,...,...,...,...
ENSG00000198695,chrMT,14672,14673,ENSG00000198695,MT-ND6
ENSG00000210194,chrMT,14741,14742,ENSG00000210194,MT-TE
ENSG00000198727,chrMT,14746,14747,ENSG00000198727,MT-CYB


In [127]:
bed_template_df["gene_id"]

gene_name
DDX11L1        ENSG00000223972
MIR6859-1      ENSG00000278267
MIR1302-2HG    ENSG00000243485
WASH7P         ENSG00000227232
MIR1302-2      ENSG00000284332
                    ...       
MT-ND6         ENSG00000198695
MT-TE          ENSG00000210194
MT-CYB         ENSG00000198727
MT-TT          ENSG00000210195
MT-TP          ENSG00000210196
Name: gene_id, Length: 60726, dtype: object

In [148]:
bed_template_df.to_csv("/sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/input/reference_data/Homo_sapiens.GRCh38.103.chr.reformatted.collapse_only.gene.region_list","\t")

In [149]:
genotype_list = pd.read_csv("/sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/output/Genotype/snmulti_QTL_analysis_psychAD_ADSP.leftnorm.bcftools_qc.plink_qc.genotype_by_chrom_files.txt","\t")

/opt/conda/envs/TensorQTL/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3508: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


In [151]:
genotype_list["#chr"]= [f'chr{x}' for x in genotype_list["#id"]]

In [152]:
genotype_list

,#id,#path,#chr
0,7,/sc/arion/projects/CommonMind/roussp01a/snmult...,chr7
1,15,/sc/arion/projects/CommonMind/roussp01a/snmult...,chr15
2,10,/sc/arion/projects/CommonMind/roussp01a/snmult...,chr10
3,2,/sc/arion/projects/CommonMind/roussp01a/snmult...,chr2
4,4,/sc/arion/projects/CommonMind/roussp01a/snmult...,chr4
5,22,/sc/arion/projects/CommonMind/roussp01a/snmult...,chr22
6,9,/sc/arion/projects/CommonMind/roussp01a/snmult...,chr9
7,17,/sc/arion/projects/CommonMind/roussp01a/snmult...,chr17
8,12,/sc/arion/projects/CommonMind/roussp01a/snmult...,chr12
9,8,/sc/arion/projects/CommonMind/roussp01a/snmult...,chr8


In [153]:
bed_template_df.merge(genotype_list)[["gene_id","#path"]].rename({"gene_id": "#id"  }, axis = 1).to_csv("/sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/output/Genotype/snmulti_QTL_analysis_psychAD_ADSP.leftnorm.bcftools_qc.plink_qc.genotype_by_chrom_files.region_list.txt","\t",index = False)

In [169]:
bed_template_df.to_csv("/sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/input/reference_data/Homo_sapiens.GRCh38.103.chr.reformatted.collapse_only.gene.unique.region_list","\t")

In [180]:
bed_template_df[]

,#chr,start,end,gene_id,gene_name
gene_id,,,,,
ENSG00000223972,chr1,11868,11869,ENSG00000223972,DDX11L1
ENSG00000278267,chr1,17435,17436,ENSG00000278267,MIR6859-1
ENSG00000243485,chr1,29553,29554,ENSG00000243485,MIR1302-2HG
ENSG00000227232,chr1,29569,29570,ENSG00000227232,WASH7P
ENSG00000284332,chr1,30365,30366,ENSG00000284332,MIR1302-2
...,...,...,...,...,...
ENSG00000198695,chrMT,14672,14673,ENSG00000198695,MT-ND6
ENSG00000210194,chrMT,14741,14742,ENSG00000210194,MT-TE
ENSG00000198727,chrMT,14746,14747,ENSG00000198727,MT-CYB


In [172]:
b = pd.read_csv("/sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/MMQTL/Test_data_for_mmQTL/gene_location_for_simulated_phenotype","\t")

/opt/conda/envs/TensorQTL/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3508: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


In [173]:
b

,1,53679771,53686289,ENSG00000162384
0,1,220141943,220220000,ENSG00000136628
1,1,229652329,229694442,ENSG00000135776
2,1,160370376,160398468,ENSG00000162738
3,1,209955661,209957904,ENSG00000162757
4,1,206680879,206762616,ENSG00000136653
...,...,...,...,...
794,1,151372010,151374420,ENSG00000159377_A2
795,1,183441351,183567381,ENSG00000116698_A4
796,1,65713902,65881552,ENSG00000116675_A4
797,1,222731244,222763275,ENSG00000143498_A3


In [226]:
a = pd.read_csv("../working/test.csv",",")

/opt/conda/envs/TensorQTL/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3508: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


In [227]:
a

,Unnamed: 0,#id,#path,#dir1_x,#dir1_y,phenotype_names
0,chr12:214119-214120,ENSG00000111181,/sc/arion/projects/CommonMind/roussp01a/snmult...,NaN,recipe_testing/data_preprocessing/Astro.PFC/ph...,62
1,chr12:262872-262873,ENSG00000010379,/sc/arion/projects/CommonMind/roussp01a/snmult...,recipe_testing/data_preprocessing/Astro.EC/phe...,recipe_testing/data_preprocessing/Astro.PFC/ph...,"63','62"
2,chr12:389272-389273,ENSG00000120647,/sc/arion/projects/CommonMind/roussp01a/snmult...,recipe_testing/data_preprocessing/Astro.EC/phe...,recipe_testing/data_preprocessing/Astro.PFC/ph...,"63','62"
3,chr12:389319-389320,ENSG00000073614,/sc/arion/projects/CommonMind/roussp01a/snmult...,recipe_testing/data_preprocessing/Astro.EC/phe...,recipe_testing/data_preprocessing/Astro.PFC/ph...,"63','62"
4,chr12:459938-459939,ENSG00000139044,/sc/arion/projects/CommonMind/roussp01a/snmult...,recipe_testing/data_preprocessing/Astro.EC/phe...,NaN,63
5,chr12:663778-663779,ENSG00000171840,/sc/arion/projects/CommonMind/roussp01a/snmult...,recipe_testing/data_preprocessing/Astro.EC/phe...,NaN,63
6,chr12:752578-752579,ENSG00000060237,/sc/arion/projects/CommonMind/roussp01a/snmult...,recipe_testing/data_preprocessing/Astro.EC/phe...,recipe_testing/data_preprocessing/Astro.PFC/ph...,"63','62"
7,chr12:990052-990053,ENSG00000002016,/sc/arion/projects/CommonMind/roussp01a/snmult...,recipe_testing/data_preprocessing/Astro.EC/phe...,recipe_testing/data_preprocessing/Astro.PFC/ph...,"63','62"
8,chr12:990508-990509,ENSG00000082805,/sc/arion/projects/CommonMind/roussp01a/snmult...,recipe_testing/data_preprocessing/Astro.EC/phe...,recipe_testing/data_preprocessing/Astro.PFC/ph...,"63','62"
9,chr12:991189-991190,ENSG00000250132,/sc/arion/projects/CommonMind/roussp01a/snmult...,recipe_testing/data_preprocessing/Astro.EC/phe...,recipe_testing/data_preprocessing/Astro.PFC/ph...,"63','62"


In [214]:
bed_template_df = load_bed_template('/sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/input/reference_data/Homo_sapiens.GRCh38.103.chr.reformatted.collapse_only.gene.gtf', "gene_name")

In [215]:
bed_template_df

,#chr,start,end,gene_id,gene_name
gene_name,,,,,
DDX11L1,chr1,11868,11869,ENSG00000223972,DDX11L1
MIR6859-1,chr1,17435,17436,ENSG00000278267,MIR6859-1
MIR1302-2HG,chr1,29553,29554,ENSG00000243485,MIR1302-2HG
WASH7P,chr1,29569,29570,ENSG00000227232,WASH7P
MIR1302-2,chr1,30365,30366,ENSG00000284332,MIR1302-2
...,...,...,...,...,...
MT-ND6,chrMT,14672,14673,ENSG00000198695,MT-ND6
MT-TE,chrMT,14741,14742,ENSG00000210194,MT-TE
MT-CYB,chrMT,14746,14747,ENSG00000198727,MT-CYB


In [228]:
genoFile = a

In [229]:
file_inv = genoFile.drop(columns = ["#id", "phenotype_names"]).to_dict("split")
file_inv['data'] = [[value for value in sublist if not pd.isna(value)] for sublist in file_inv['data']] 


## There will alwayse be genotype file due to left join,
## There will alwayse be covar file as len(covFile) must == len(PhenoFile), and covar column is the same string accross all rows
## So only if there is no bed.gz there will be problem.
regional_data = {"data":file_inv["data"],"meta_info": genoFile[["#id","phenotype_names"]].reset_index().to_dict("split")['data'] }

# Recreate file_inv based on the filtered genoFile
file_inv = genoFile.drop(columns=["#id", "phenotype_names"]).to_dict("split")
file_inv['data'] = [[value for value in sublist if not pd.isna(value)] for sublist in file_inv['data']] 

# Recreate the regional_data based on the filtered data
regional_data = {"data": file_inv["data"],
                 "meta_info": genoFile[["#id", "phenotype_names"]].reset_index().to_dict("split")['data']}

In [232]:
len(regional_data["data"])

11

In [233]:
len(regional_data["meta_info"])

11

In [234]:
meta_info = regional_data["meta_info"]

In [235]:
len(meta_info)

11

In [239]:
pd.DataFrame(meta_info,regional_data["data"])

ValueError: all arrays must be same length

In [96]:
pos_col = [col for col in a.columns if col.startswith('pos')]
a.index = pd.Series(a[pos_col].values.flatten()).dropna().unique()

ValueError: Length mismatch: Expected axis has 14522 elements, new values have 14462 elements

In [100]:
pd.Series(a[pos_col].values.flatten()).dropna().unique()

array(['chr1:173861-173862;ENSG00000241860',
       'chr1:778746-778747;ENSG00000237491',
       'chr1:825137-825138;ENSG00000228794', ...,
       'chr21:46458890-46458891;ENSG00000160305',
       'chr21:46605207-46605208;ENSG00000160307',
       'chr21:46635594-46635595;ENSG00000160310'], dtype=object)

In [104]:
a["pos1_y"].dropna()

0             chr1:173861-173862;ENSG00000241860
1             chr1:778746-778747;ENSG00000237491
2             chr1:825137-825138;ENSG00000228794
3             chr1:923922-923923;ENSG00000187634
4             chr1:959308-959309;ENSG00000188976
                          ...                   
14517    chr21:46323874-46323875;ENSG00000160298
14518    chr21:46324140-46324141;ENSG00000160299
14519    chr21:46458890-46458891;ENSG00000160305
14520    chr21:46605207-46605208;ENSG00000160307
14521    chr21:46635594-46635595;ENSG00000160310
Name: pos1_y, Length: 14021, dtype: object

In [105]:
a["pos1_x"].dropna()

0             chr1:173861-173862;ENSG00000241860
1             chr1:778746-778747;ENSG00000237491
2             chr1:825137-825138;ENSG00000228794
3             chr1:923922-923923;ENSG00000187634
4             chr1:959308-959309;ENSG00000188976
                          ...                   
14517    chr21:46323874-46323875;ENSG00000160298
14518    chr21:46324140-46324141;ENSG00000160299
14519    chr21:46458890-46458891;ENSG00000160305
14520    chr21:46605207-46605208;ENSG00000160307
14521    chr21:46635594-46635595;ENSG00000160310
Name: pos1_x, Length: 14024, dtype: object

In [106]:
a["pos3_y"].dropna()

17          chr1:1399334-1399335;ENSG00000221978
32          chr1:1891116-1891117;ENSG00000078369
35          chr1:2228318-2228319;ENSG00000157933
67          chr1:6785453-6785454;ENSG00000171735
69          chr1:7784319-7784320;ENSG00000049246
                          ...                   
14503    chr21:44873902-44873903;ENSG00000183255
14505    chr21:45073852-45073853;ENSG00000197381
14513    chr21:46228823-46228824;ENSG00000160285
14519    chr21:46458890-46458891;ENSG00000160305
14521    chr21:46635594-46635595;ENSG00000160310
Name: pos3_y, Length: 2694, dtype: object

In [107]:
a["pos3_x"].dropna()

0             chr1:173861-173862;ENSG00000241860
1             chr1:778746-778747;ENSG00000237491
2             chr1:825137-825138;ENSG00000228794
3             chr1:923922-923923;ENSG00000187634
4             chr1:959308-959309;ENSG00000188976
                          ...                   
14517    chr21:46323874-46323875;ENSG00000160298
14518    chr21:46324140-46324141;ENSG00000160299
14519    chr21:46458890-46458891;ENSG00000160305
14520    chr21:46605207-46605208;ENSG00000160307
14521    chr21:46635594-46635595;ENSG00000160310
Name: pos3_x, Length: 13633, dtype: object

In [113]:
sum(a.drop(columns=['#id',"#path"]).isna().sum(axis=1))

54864

In [116]:
a.drop(columns=['#id',"#path"]).isna().sum(axis=1).describe()

count    14522.000000
mean         3.777992
std          2.474082
min          0.000000
25%          4.000000
50%          4.000000
75%          4.000000
max         12.000000
dtype: float64

In [123]:
len(set(a["#id"]))

14462

In [194]:
import pandas as pd
import qtl.io
from pathlib import Path

def prepare_bed(df, bed_template_df, chr_subset=None):
    bed_df = pd.merge(bed_template_df, df, left_index=True, right_index=True)
    bed_df = bed_df.groupby('#chr', sort=False, group_keys=False).apply(lambda x: x.sort_values('start'))
    if chr_subset is not None:
        bed_df = bed_df[bed_df.chr.isin(chr_subset)]
    return bed_df

def load_and_preprocess_data(input_path, drop_columns):
    df = pd.read_csv(input_path, sep="	", skiprows=0)
    dc = [col for col in df.columns if col in drop_columns] # Take interscet between df.columns and drop_columns
    df = df.drop(dc,axis = 1) # drop the intersect
    if len(df.columns) < 2:
        raise ValueError("There are too few columns in the loaded dataframe, please check the delimiter of the input file. The default delimiter is tab")
    return df

def rename_samples_using_lookup(df, lookup_path):
    sample_participant_lookup = Path(lookup_path)
    if sample_participant_lookup.is_file():
        sample_participant_lookup_s = pd.read_csv(sample_participant_lookup, sep="\t", index_col=1, dtype={0:str,1:str})
        df.rename(columns=sample_participant_lookup_s.to_dict()["genotype_id"], inplace=True)
    return df

def load_bed_template(input_path, phenotype_id_type):
    if sum(qtl.io.gtf_to_tss_bed(input_path, feature='gene', phenotype_id = "gene_id").index.duplicated()) > 0:
        raise valueerror(f"gtf file {input_path} needs to be collapsed into gene model by reference data processing module")

    bed_template_df_id = qtl.io.gtf_to_tss_bed(input_path, feature='transcript', phenotype_id="gene_id")
    bed_template_df_name = qtl.io.gtf_to_tss_bed(input_path, feature='transcript', phenotype_id="gene_name")
    bed_template_df = bed_template_df_id.merge(bed_template_df_name, on=["chr", "start", "end"])
    bed_template_df.columns = ["#chr", "start", "end", "gene_id", "gene_name"]
    bed_template_df = bed_template_df.set_index(phenotype_id_type, drop=False)

    return bed_template_df

In [208]:
df = load_and_preprocess_data("/hpc/users/sunh14/snmulti_QTL/output/molecular_phenotype/ATAC_dreamlet/pb4dreamlet_subclass_ATAC.SMC.PFC.low_expression_filtered.normalized.log2cpm.gct.gz", ["#chr","chr", "start", "end","stop","annot.seqnames","annot.start","annot.end"]) # This function will drop any columns from the DF based on the drop_columns list, so it is good to make the list comprehensive to accomodate different source of inputs.    df.set_index(df.columns[0], inplace=True)
df.set_index(df.columns[0], inplace=True)
df = rename_samples_using_lookup(df, "/sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/input/snmulti_QTL_sample_participant_lookup_re")
bed_template_df = pd.read_csv("/sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/input/Peak_cell_anno_region_list",sep = "\t").set_index("ID")


In [217]:
bed_template_df = pd.read_csv("/sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/input/Peak_cell_anno_region_list",sep = "\t").set_index("ID")


In [220]:
bed_template_df = bed_template_df.assign(ID = bed_template_df.index )

In [221]:
bed_df = prepare_bed(df, bed_template_df)


In [222]:
bed_df

,#chr,start,end,ID,G-MSBB-MB000167-BR-MSBB-71808,G-MSBB-MB000201-BR-MSBB-71848,G-MSBB-MB000269-BR-MSBB-71927,G-MSBB-MB000055-BR-MSBB-71689,G-MSBB-MB000311-BR-MSBB-71972,G-MSBB-MB000165-BR-MSBB-71806,...,G-MSBB-MB000042-BR-MSBB-70672,G-MSBB-MB000151-BR-MSBB-71792,G-MSBB-MB000099-BR-MSBB-71736,G-MSBB-MB000325-BR-MSBB-71986,G-MSBB-MB000182-BR-MSBB-71828,G-MSBB-MB000318-BR-MSBB-71979,G-MSBB-MB000041-BR-MSBB-70665,208754,X565,37563
chr2:150279268-150279768,chr2,150279268,150279768,chr2:150279268-150279768,7.500654,7.958824,7.960965,8.521975,8.124048,6.066773,...,8.302691,8.538269,7.998865,3.667575,7.252062,8.323700,7.481947,7.995917,7.756237,7.769193
chr2:160306412-160306912,chr2,160306412,160306912,chr2:160306412-160306912,6.105146,8.304926,7.326377,7.640177,7.509041,6.923105,...,7.475810,8.383594,7.363166,7.956301,7.494912,7.703955,7.481947,7.306050,6.838632,7.769193
chr7:96329208-96329708,chr7,96329208,96329708,chr7:96329208-96329708,6.965438,8.379874,6.166354,7.997324,6.775259,6.557563,...,5.331189,8.922314,7.363166,6.162309,6.959891,7.496142,6.088755,7.691831,3.667568,7.381899
chr7:150720416-150720916,chr7,150720416,150720916,chr7:150720416-150720916,7.890160,8.583854,7.570247,7.887813,7.303325,6.923105,...,6.083383,7.785198,6.199281,8.118001,7.702711,6.961080,8.032315,7.511798,7.756237,8.325868


In [200]:
bed_df = bed_df.rename(columns={'gene_id': 'ID'})

In [202]:
bed_df.reset_index()

,index,#chr,start,end,G-MSBB-MB000167-BR-MSBB-71808,G-MSBB-MB000201-BR-MSBB-71848,G-MSBB-MB000269-BR-MSBB-71927,G-MSBB-MB000055-BR-MSBB-71689,G-MSBB-MB000311-BR-MSBB-71972,G-MSBB-MB000165-BR-MSBB-71806,...,G-MSBB-MB000042-BR-MSBB-70672,G-MSBB-MB000151-BR-MSBB-71792,G-MSBB-MB000099-BR-MSBB-71736,G-MSBB-MB000325-BR-MSBB-71986,G-MSBB-MB000182-BR-MSBB-71828,G-MSBB-MB000318-BR-MSBB-71979,G-MSBB-MB000041-BR-MSBB-70665,208754,X565,37563
0,chr2:150279268-150279768,chr2,150279268,150279768,7.500654,7.958824,7.960965,8.521975,8.124048,6.066773,...,8.302691,8.538269,7.998865,3.667575,7.252062,8.323700,7.481947,7.995917,7.756237,7.769193
1,chr2:160306412-160306912,chr2,160306412,160306912,6.105146,8.304926,7.326377,7.640177,7.509041,6.923105,...,7.475810,8.383594,7.363166,7.956301,7.494912,7.703955,7.481947,7.306050,6.838632,7.769193
2,chr7:96329208-96329708,chr7,96329208,96329708,6.965438,8.379874,6.166354,7.997324,6.775259,6.557563,...,5.331189,8.922314,7.363166,6.162309,6.959891,7.496142,6.088755,7.691831,3.667568,7.381899
3,chr7:150720416-150720916,chr7,150720416,150720916,7.890160,8.583854,7.570247,7.887813,7.303325,6.923105,...,6.083383,7.785198,6.199281,8.118001,7.702711,6.961080,8.032315,7.511798,7.756237,8.325868


In [212]:
bed_df = prepare_bed(df, bed_template_df)

In [223]:
bed_df.drop_duplicates("ID", keep=False)

,#chr,start,end,ID,G-MSBB-MB000167-BR-MSBB-71808,G-MSBB-MB000201-BR-MSBB-71848,G-MSBB-MB000269-BR-MSBB-71927,G-MSBB-MB000055-BR-MSBB-71689,G-MSBB-MB000311-BR-MSBB-71972,G-MSBB-MB000165-BR-MSBB-71806,...,G-MSBB-MB000042-BR-MSBB-70672,G-MSBB-MB000151-BR-MSBB-71792,G-MSBB-MB000099-BR-MSBB-71736,G-MSBB-MB000325-BR-MSBB-71986,G-MSBB-MB000182-BR-MSBB-71828,G-MSBB-MB000318-BR-MSBB-71979,G-MSBB-MB000041-BR-MSBB-70665,208754,X565,37563
chr2:150279268-150279768,chr2,150279268,150279768,chr2:150279268-150279768,7.500654,7.958824,7.960965,8.521975,8.124048,6.066773,...,8.302691,8.538269,7.998865,3.667575,7.252062,8.323700,7.481947,7.995917,7.756237,7.769193
chr2:160306412-160306912,chr2,160306412,160306912,chr2:160306412-160306912,6.105146,8.304926,7.326377,7.640177,7.509041,6.923105,...,7.475810,8.383594,7.363166,7.956301,7.494912,7.703955,7.481947,7.306050,6.838632,7.769193
chr7:96329208-96329708,chr7,96329208,96329708,chr7:96329208-96329708,6.965438,8.379874,6.166354,7.997324,6.775259,6.557563,...,5.331189,8.922314,7.363166,6.162309,6.959891,7.496142,6.088755,7.691831,3.667568,7.381899
chr7:150720416-150720916,chr7,150720416,150720916,chr7:150720416-150720916,7.890160,8.583854,7.570247,7.887813,7.303325,6.923105,...,6.083383,7.785198,6.199281,8.118001,7.702711,6.961080,8.032315,7.511798,7.756237,8.325868


In [225]:
qtl.io.write_bed(bed_df, "test_atac_output.bed.gz")

/bin/sh: 1: bgzip: not found


BrokenPipeError: [Errno 32] Broken pipe

In [ ]:
bed_df = prepare_bed(df, bed_template_df)
bed_df = bed_df.rename(columns={'gene_id': 'ID'})
qtl.io.write_bed(bed_df, "test_atac_output")
bed_df[["#chr","start","end","ID"]].assign(path = ${_output[0]:r}).to_csv(${_output[1]:r},"\t",index = False) 


In [241]:
data = regional_data["data"]

In [242]:
zip(meta_info, data)

In [245]:
genoFile.to_dict("records")

[{'Unnamed: 0': 'chr12:214119-214120',
  '#id': 'ENSG00000111181',
  '#path': '/sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/output/Genotype/snmulti_QTL_analysis_psychAD_ADSP.leftnorm.bcftools_qc.plink_qc.12.bed',
  '#dir1_x': nan,
  '#dir1_y': 'recipe_testing/data_preprocessing/Astro.PFC/phenotype_data/pb4dreamlet_subclass.Astro.PFC.low_expression_filtered.normalized.log2cpm.gct.cov_pca_factor.residual.bed.chr12.bed.gz',
  'phenotype_names': '62'},
 {'Unnamed: 0': 'chr12:262872-262873',
  '#id': 'ENSG00000010379',
  '#path': '/sc/arion/projects/CommonMind/roussp01a/snmulti_QTL/output/Genotype/snmulti_QTL_analysis_psychAD_ADSP.leftnorm.bcftools_qc.plink_qc.12.bed',
  '#dir1_x': 'recipe_testing/data_preprocessing/Astro.EC/phenotype_data/pb4dreamlet_subclass.Astro.EC.low_expression_filtered.normalized.log2cpm.gct.cov_pca_factor.residual.bed.chr12.bed.gz',
  '#dir1_y': 'recipe_testing/data_preprocessing/Astro.PFC/phenotype_data/pb4dreamlet_subclass.Astro.PFC.low_expression_filtered.